In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
import time
import copy
import linecache
import gzip
import pickle
import itertools
from sklearn.preprocessing import MinMaxScaler
import datetime
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
from torch.nn import functional as F

import sys, argparse, os
import numpy as np
import random
import sys, argparse, os

import torch.nn.functional as F
import torch.utils.data as torchdata

sys.path.insert(0, '../')
sys.path.append("/scratch2/ml_flood/mlflood/")
from pathlib import Path
from mlflood.conf import PATH_DATA
from mlflood.conf import rain_const, waterdepth_diff_const
import h5py
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from mlflood.utils import new_log
from models.utae import UTAE
from models.CNNrolling import CNNrolling
from models.unet3d import UNet3D
from models.unet import UNet
from dataset_utae import load_test_dataset, dataloader_args_utae_test

from training import *
#from developing_suite4 import *

In [ ]:
path_exp = "/scratch2/ml_flood/data/checkpoints/709/cluster/utae_L1/experiment_0/"
path_exp = "/scratch2/ml_flood/data/checkpoints/709/utae_temp/experiment_6/"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)
args = []   

In [ ]:
parser = argparse.ArgumentParser(description="evaluation")
parser.add_argument("--catchment_kwargs", default='./default_catchment_kwargs.yml', type=str, 
                    help="path to catchment kwargs saved in yml file")

exp_hp = ['--catchment_kwargs=../mlflood/exp_yml/exp_8_utae_temp.yml']
args = parser.parse_args(exp_hp)

with open(args.catchment_kwargs) as file:
    catchment_kwargs = yaml.full_load(file)
    
# Always in eval mode
catchment_kwargs['fix_indexes'] = True 

In [ ]:
dataloaders = {}
dataset = load_test_dataset(catchment_kwargs)
                                                             
dataloaders["test"] = dataloader_args_utae_test(dataset, catchment_num = catchment_kwargs['num'])
dataset_test = dataloaders["test"]

In [ ]:
model = UTAE(args)  # You don't need args in UTAE
file_path1 = path_exp + "model.pth.tar"
model.load_state_dict(torch.load(file_path1))
model.cuda()

In [ ]:
from evaluation import predict_event 

event_num = 0
start_ts = None    # start t_s should be 5 since timestep = 5 (it should have returned an error, why not)?
start_ts = 5
# timestep = 12

pred_cnn, gt_cnn, mask_cnn = predict_event(model, dataset, event_num, start_ts, ar = False)  

In [ ]:
from evaluation import numpy2movie 

animation = numpy2movie(pred_cnn, gt_cnn)
animation.ipython_display(fps=10, loop=True, autoplay=True)

In [ ]:
zoom = [500,1000,500,1000]
save_folder = "/scratch2/ml_flood/data/checkpoints/709/cluster/utae_L1/experiment_0/results/c_recons/"
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=None, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
zoom = [500,1000,500,1000]
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=zoom, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
zoom = [1500,2000,0,500]
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=zoom, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
zoom = [2000,2500,1000,2000]
save_folder = "/scratch2/ml_flood/data/checkpoints/709/cluster/utae_L1/experiment_0/results/c_recons/"
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=zoom, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
from evaluation import mae_event, plot_maes

maes = mae_event(pred_cnn, gt_cnn)
print(maes.shape)
plot_maes(maes, ["utae"],  title = f"MAE for {1+12} ts ahead")

In [ ]:
len(maes)

In [ ]:
pred_cnn.shape

In [ ]:
a = abs(pred_cnn-gt_cnn)
mae_ev = []
for i in range(43):
    t = np.mean(a[i,:,:])
    mae_ev.append(t)
len(mae_ev)

In [ ]:
t = np.arange(len(mae_ev))
plt.plot(t, mae_ev, label="utae")
plt.legend()
#plt.tight_layout()
plt.ylabel('Absolute Error (cm)')
plt.xlabel('Timesteps')
# filename = save_folder + 'Mae_' + "ev0" +'.png'
# plt.savefig(filename, dpi=1200)

In [ ]:
print(mae_ev)